In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import warnings
import tika
from tika import parser
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import re
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
warnings.filterwarnings('ignore')



In [2]:
data = ['data/Ml_1.pdf', 'data/Ml_2.pdf', 'data/Ml_3.pdf', 'data/Ml_4.pdf', 'data/Ml_5.pdf', 'data/Ml_6.pdf', 'data/Ml_7.pdf']
biblioteca = []
tika.initVM()

for i in range(0, len(data)):
    
    parsed = parser.from_file(data[i])
    metadados = parsed["metadata"]
    texto = parsed["content"]
    biblioteca.append(texto)

In [3]:
print(len(biblioteca))

7


In [4]:
# f = open ("artigo2.txt", "w")
# f.write (str(texto))
# f.close ()

In [5]:

# f = open ("artigo2.txt", "r")
# texto = f.read()

In [6]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def split_pdf(string): 
    li = list(string.split(" ")) 
    return li

In [8]:
def first_clear(data):
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    #remove links and mark
    data = [re.sub('(https:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(http:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(www.:\S+)|(@)|(¿)', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # remove spaces in begining and end
    data = [re.sub(' +', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    return data

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [11]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
def convert(s): 
    a = 0
    # initialization of string to "" 
    new = "" 
  
    # traverse in the string  
    for x in s: 
        new += x + ' '
        a+=1
  
    # return string  
    return new, a

In [13]:

data_cleared = []
train_cleared = []

In [14]:
total_char = 0
for i in range(0, len(biblioteca)):
    texto = biblioteca[i]
    data = split_pdf(texto)
    data = first_clear(data)
    d, a = convert(data)
    
    total_char += a
    
    if i < 5:
        data_cleared.append(d)
    else:
        train_cleared.append(d)

In [15]:
print(len(data_cleared))
print(len(train_cleared))
print(total_char)

5
2
40061


In [16]:
tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(data_cleared)]

In [17]:
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
#     print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

Model Saved


In [18]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data

for i in range(0, len(train_cleared)):
    test_data = word_tokenize(train_cleared[i])
    v1 = model.infer_vector(test_data)

    # to find most similar doc using tags
    similar_doc = model.docvecs.most_similar([v1])
    print(similar_doc)


[('1', 0.951371431350708), ('0', 0.8684254288673401), ('3', 0.8545146584510803), ('4', 0.8541525602340698), ('2', 0.8390761613845825)]
[('3', 0.3825182616710663), ('2', 0.2993747591972351), ('4', 0.2675929367542267), ('0', 0.22543717920780182), ('1', 0.2105032503604889)]
